In [1]:
import altair as alt
import numpy as np
import pandas as pd
import panel as pn
#pn.extension('vega')
#pn.extension('tabulator')

# Read in the Excel file with Routine Core Analysis (RCA), image file names and Thomeer Capillary Pressure parameters:

In [2]:
#read the file
file = r'Clastic_TS_Altair7.xlsx'
source7 = pd.read_excel(file,index_col=False)


In [3]:
source7.head()

,Sample,Porosity,Permeability,sqrt(k/phi),lPerm,image,Mode,BV1,G1,Pd1,ROCK_INDEX
0,1,0.292,1770.0,77.86,3.25,./data/1.png,46.33,29.2,0.19,3.76,1
1,2,0.310,2200.0,84.24,3.34,./data/Great_SS.png,50.38,31.0,0.18,3.51,1
2,3,0.218,28.4,11.41,1.45,./data/12.png,9.41,21.8,0.39,14.32,2
3,4,0.144,23.0,12.64,1.36,./data/MediumSS.png,8.68,14.4,0.40,15.33,2
4,5,0.159,4.9,5.55,0.69,./data/DistalTurbidite.png,4.78,15.9,0.47,25.27,2


# Basic Porosity vs. Permeability Cross Plot that show Thin Section Images and Selected Data:
**Colored by the sqrt(k/phi)**

In [4]:
# Brush for selection
brush = alt.selection(type='interval')



chart = alt.Chart(source7).mark_circle(size=400).encode(
    alt.X('Porosity:Q',
        scale=alt.Scale(domain=(0, 0.4))),
    #y='Perm',
    alt.Y('Permeability:Q',scale=alt.Scale(type='log', base=10,domain=(.0001, 10000))),
    #color='LPerm:Q',
    color = alt.Color('sqrt(k/phi):Q', scale=alt.
                      Scale(scheme = 'rainbow'),legend=None),
    #alt.Y('LPerm:Q',scale=alt.Scale(type='log', base=10,domain=(.0001, 10000))),
    tooltip=['image','Sample','Porosity','Permeability','sqrt(k/phi)'],  # Must be a list for the image to render
    
).properties(
    width=500,
    height=500, 

    title='Porosity vs. Permeability Cross Plot: hover over Sample to see Thin Section',

).add_selection(brush)
    
vega_pane = pn.pane.Vega(chart.add_selection(
    brush
))
    

    


# Base chart for data tables
ranked_text = alt.Chart(source7).mark_text(align='right').encode(
    y=alt.Y('row_number:O',axis=None)
).transform_window(
    row_number='row_number()'
).transform_filter(
    brush
).transform_window(
    rank='rank(row_number)'
).transform_filter(
    alt.datum.rank<30
)


# Data Tables
Porosity = ranked_text.encode(text='Porosity:N').properties(title=alt.TitleParams(text='Porosity', align='right'))
Permeability = ranked_text.encode(text='Permeability:N').properties(title=alt.TitleParams(text='Permeability', align='right'))
Sample = ranked_text.encode(text='Sample:N').properties(title=alt.TitleParams(text='Sample', align='right'))
sqrt = ranked_text.encode(text='sqrt(k/phi):N').properties(title=alt.TitleParams(text ='sqrt(k/phi)', align='right'))
Image = ranked_text.encode(tooltip ='image').properties(title=alt.TitleParams(text ='image', align='right'))





text = alt.hconcat(Sample, Porosity, Permeability,sqrt,Image) # Combine data tables

# Build chart
alt.hconcat(
    chart,
    text
).resolve_legend(
    color="independent"
).configure_view(strokeWidth=0)




alt.HConcatChart(...)

# Create Pc curves from Thomeer Parameters:

In [5]:
# =============================================================================
# # ===========================================================================
# # #--------------------------------------------------------------------------
# # #           
# # #               CREATE Pc PLOTS from kNN Values of Thomeer
# # #                   
# # #--------------------------------------------------------------------------
# # ===========================================================================
# =============================================================================

Pd1 = source7['Pd1']
G1  = source7['G1']
BV1 = source7['BV1']



Pc = 0.5
bvarray = []; #make list of 0 length
pcarray = []
Sample  = []

#for k in range (0,len(source6)):
   
for k in range (0,len(source7)):
    Pc = 0.5    
    for j in range(1, 105, 1):
        if Pc > Pd1[k]:
            BVOCC1 = BV1[k] * 10**((-0.434 * G1[k]) / np.log10(Pc / Pd1[k]))
        else:
            BVOCC1 = 0.001



        BVOCC = BVOCC1
        
        Sample.append(k+1)
        bvarray.append(BVOCC); #add items 
        pcarray.append(Pc); #add items 

        Pc = Pc * 1.12

   

**Create DataFrame of Pc data**

In [6]:
df_bv=pd.DataFrame(bvarray,columns=['BVOCC'])
df_pc=pd.DataFrame(pcarray, columns=['Pc'])
df_sample=pd.DataFrame(Sample, columns=['Sample'])
#index_col=False

df_Pc=pd.DataFrame()
df_Pc['Sample'] = df_sample
df_Pc['Pc'] = df_pc
df_Pc['BVOCC'] = df_bv


df_Pc.head()
#print(len(df_Pc))

,Sample,Pc,BVOCC
0,1,0.500000,0.001
1,1,0.560000,0.001
2,1,0.627200,0.001
3,1,0.702464,0.001
4,1,0.786760,0.001


**Merge Pc data with RCA to make a single DataFrame**

In [7]:
# reading the files
f1 = df_Pc
f2 = source7
  
# merging the files
f3 = f1[["Sample", 
         "Pc",
         "BVOCC"]].merge(f2[["Sample"  , 
                              "Porosity"  , 
                              "Permeability" , 
                              "sqrt(k/phi)" , 
                              "lPerm" , 
                              "image" , 
                              "Mode" , 
                              "BV1" , 
                              "G1" , 
                              "Pd1" , 
                              "ROCK_INDEX"]] , 
                               on = "Sample", 
                               how = "left")
  
# creating a new file
f3.to_excel("Clastic_Pc.xlsx", index = False)

f3.head()

,Sample,Pc,BVOCC,Porosity,Permeability,sqrt(k/phi),lPerm,image,Mode,BV1,G1,Pd1,ROCK_INDEX
0,1,0.500000,0.001,0.292,1770.0,77.86,3.25,./data/1.png,46.33,29.2,0.19,3.76,1
1,1,0.560000,0.001,0.292,1770.0,77.86,3.25,./data/1.png,46.33,29.2,0.19,3.76,1
2,1,0.627200,0.001,0.292,1770.0,77.86,3.25,./data/1.png,46.33,29.2,0.19,3.76,1
3,1,0.702464,0.001,0.292,1770.0,77.86,3.25,./data/1.png,46.33,29.2,0.19,3.76,1
4,1,0.786760,0.001,0.292,1770.0,77.86,3.25,./data/1.png,46.33,29.2,0.19,3.76,1


# Make our final plot of Porosity-Permeability Cross Plot, Select RCA data and Selected Pc curves:

In [8]:
# Brush for selection
brush = alt.selection(type='interval')



chart = alt.Chart(source7).mark_circle(size=300).encode(
    alt.X('Porosity:Q',
        scale=alt.Scale(domain=(0, 0.4))),
    #y='Perm',
    alt.Y('Permeability:Q',scale=alt.Scale(type='log', base=10,domain=(.0001, 10000))),
    
    color=alt.condition( brush,'ROCK_INDEX:O', alt.value('lightgray'),
                        scale=alt.Scale(
                        domain=[ 1,         2,         3  ,  4 ],
                        range =['cyan', 'blue'  , 'yellow', 'brown'])),
    
    
    
    
    
    #alt.Y('LPerm:Q',scale=alt.Scale(type='log', base=10,domain=(.0001, 10000))),
    tooltip=['image','Sample','Porosity','Permeability','sqrt(k/phi)'],  # Must be a list for the image to render
    
).properties(
    width=350,
    height=450, 

    title='Porosity vs. Permeability Cross Plot Colored by PRT',

).add_selection(brush)
    
vega_pane = pn.pane.Vega(chart.add_selection(
    brush
))


# Base chart for data tables
ranked_text = alt.Chart(source7).mark_text(align='right').encode(
    y=alt.Y('row_number:O',axis=None)
).transform_window(
    row_number='row_number()'
).transform_filter(
    brush
).transform_window(
    rank='rank(row_number)'
).transform_filter(
    alt.datum.rank<30
)


# Data Tables
Porosity = ranked_text.encode(text='Porosity:N').properties(title=alt.TitleParams(text='Porosity', align='right'))
Permeability = ranked_text.encode(text='Permeability:N').properties(title=alt.TitleParams(text='Permeability', align='right'))
Sample = ranked_text.encode(text='Sample:N').properties(title=alt.TitleParams(text='Sample', align='right'))
sqrt = ranked_text.encode(text='sqrt(k/phi):N').properties(title=alt.TitleParams(text ='sqrt(k/phi)', align='right'))
Image = ranked_text.encode(tooltip ='image').properties(title=alt.TitleParams(text ='image', align='right'))
RI = ranked_text.encode(text='ROCK_INDEX:N').properties(title=alt.TitleParams(text='PRT', align='right'))


text = alt.hconcat(Sample, Porosity, Permeability,RI) # Combine data tables



#------------------------------------------------
#
#       Capillary Pressure Curves colored by PRT
#
#------------------------------------------------
pc = alt.Chart(f3).mark_line(clip=True).encode(
    alt.X('BVOCC:Q',
        scale=alt.Scale(domain=(40, 0))
    ),    
    alt.Y('Pc:Q',scale=alt.Scale(type='log', base=10,domain=(1, 100000))),
    color=alt.condition(brush, 'ROCK_INDEX:O', alt.value('lightgray'),scale=alt.Scale(scheme='rainbow')),
    #tooltip='Mode:Q', 
    tooltip=['image','Sample','Porosity','Permeability','sqrt(k/phi)'],
    strokeDash='Sample:N',
).properties(
    title='Pc Curves with PRT on Color Axis',
    width=350,
    height=450, 
    
).transform_filter(
    brush
)

    


# Build chart
alt.hconcat(
    chart,
    text,
    pc
).resolve_legend(
    color="shared"
).configure_view(strokeWidth=0)





alt.HConcatChart(...)

**starting code to show images with the selected RCA data:**

In [9]:
# Brush for selection
brush = alt.selection(type='interval')



chart = alt.Chart(source7).mark_circle(size=300).encode(
    alt.X('Porosity:Q',
        scale=alt.Scale(domain=(0, 0.4))),
    #y='Perm',
    alt.Y('Permeability:Q',scale=alt.Scale(type='log', base=10,domain=(.0001, 10000))),
    
    color=alt.condition( brush,'ROCK_INDEX:O', alt.value('lightgray'),
                        scale=alt.Scale(
                        domain=[ 1,         2,         3  ,  4 ],
                        range =['cyan', 'blue'  , 'yellow', 'brown'])),
    tooltip=['image','Sample','Porosity','Permeability','sqrt(k/phi)'],  # Must be a list for the image to render
    
).properties(
    width=350,
    height=450, 

    title='Porosity vs. Permeability Cross Plot Colored by PRT',

).add_selection(brush)
    
vega_pane = pn.pane.Vega(chart.add_selection(
    brush
))


# Base chart for data tables
ranked_text = alt.Chart(source7).mark_text(align='right').encode(
    y=alt.Y('row_number:O',axis=None)
).transform_window(
    row_number='row_number()'
).transform_filter(
    brush
).transform_window(
    rank='rank(row_number)'
).transform_filter(
    alt.datum.rank<30
)


# Data Tables
Porosity = ranked_text.encode(text='Porosity:N').properties(title=alt.TitleParams(text='Porosity', align='right'))
Permeability = ranked_text.encode(text='Permeability:N').properties(title=alt.TitleParams(text='Permeability', align='right'))
Sample = ranked_text.encode(text='Sample:N').properties(title=alt.TitleParams(text='Sample', align='right'))
sqrt = ranked_text.encode(text='sqrt(k/phi):N').properties(title=alt.TitleParams(text ='sqrt(k/phi)', align='right'))
Image = ranked_text.encode(tooltip ='image').properties(title=alt.TitleParams(text ='image', align='right'))
RI = ranked_text.encode(text='ROCK_INDEX:N').properties(title=alt.TitleParams(text='PRT', align='right'))


text = alt.hconcat(Sample, Porosity, Permeability,RI) # Combine data tables



#------------------------------------------------
#
#       Capillary Pressure Curves colored by PRT
#
#------------------------------------------------
pc = alt.Chart(f3).mark_line(clip=True).encode(
    alt.X('BVOCC:Q',
        scale=alt.Scale(domain=(40, 0))
    ),    
    alt.Y('Pc:Q',scale=alt.Scale(type='log', base=10,domain=(1, 100000))),
    color=alt.condition( brush,'ROCK_INDEX:O', alt.value('lightgray'),
                    scale=alt.Scale(
                    domain=[ 1,         2,         3  ,  4 ],
                    range =['cyan', 'blue'  , 'yellow', 'brown'])),
 
    #color=alt.condition(brush, 'ROCK_INDEX:O', alt.value('lightgray'),scale=alt.Scale(scheme='rainbow')),
    #tooltip='Mode:Q', 
    tooltip=['image','Sample','Porosity','Permeability','sqrt(k/phi)'],
    strokeDash='Sample:N',
).properties(
    title='Pc Curves with PRT on Color Axis',
    width=350,
    height=450, 

  
).transform_filter(
    brush
)

    


# Build chart
alt.hconcat(
    pc,
    text,
    chart
).resolve_legend(
    color="shared"
).configure_view(strokeWidth=0)




alt.HConcatChart(...)

  },
  "mark": {"type": "text", "tooltip": true},
  "encoding": {
    "text": {"field": "image"},
    "y": {"field": "image", "axis": null}
  }
}
